# Simulations light inter

## 0. Imports

In [1]:
import numpy as np
import time as t
import random
from oawidgets.plantgl import *
from openalea.plantgl.all import Material, Color3, Scene
import oawidgets.mtg

from openalea.archicrop.archicrop import ArchiCrop
from openalea.archicrop.display import build_scene, display_scene
from openalea.archicrop.simulation import read_sti_file, read_xml_file, LHS_param_sampling, params_for_curve_fit
import matplotlib.pyplot as plt
%gui qt

In [2]:
# random.seed(18)

## 1. Set management parameters

Set parameters regarding the spatial (and temporal) configuration of the crop.

In [3]:
file_tec_xml = 'Mais_tec.xml'
params_tec = ['densitesem', 'interrang']
tec_stics = read_xml_file(file_tec_xml, params_tec)
sowing_density = tec_stics['densitesem']
inter_row = 70

## 2. Format crop-scale growth and senescence dynamics

From crop-scale data either measured or simulated with a crop model, generate a dictionnary of dictionnaries for each day with the following values :
 - "Thermal time" (float): cumulated thermal time from beginning of simulation to current day (in °C.day)
 - "Phenology" (str): current phenological stage (germination, juvenile, exponential or repro)
 - "Plant leaf area" (float): plant leaf area (in cm²) 
 - "Leaf area increment" (float): leaf area increment compared to previous day (in cm²) 
 - "Plant senescent leaf area" (float): senescent plant leaf area (in cm²) 
 - "Senescent leaf area increment" (float): senescent leaf area increment compared to previous day (in cm²) 
 - "Plant height" (float): plant height (in cm) 
 - "Height increment" (float): height increment compared to previous day (in cm).

In [4]:
stics_output_file = 'mod_smaize.sti'
daily_dynamics = read_sti_file(stics_output_file, sowing_density)

# Time series 
# for thermal time, plant leaf area, plant senescent leaf area and plant height
thermal_time = [value["Thermal time"] for value in daily_dynamics.values()]
leaf_area_plant = [value["Plant leaf area"] for value in daily_dynamics.values()]
sen_leaf_area_plant = [value["Plant senescent leaf area"] for value in daily_dynamics.values()]
height_canopy = [value["Plant height"] for value in daily_dynamics.values()]

In [6]:
dates = [value["Date"] for value in daily_dynamics.values()]
dates[0]

'1996-04-21'

In [ ]:
file_plt_xml = 'corn_plt.xml'
params_sen = ['durvieF', 'ratiodurvieI']
sen_stics = read_xml_file(file_plt_xml, params_sen)
lifespan = sen_stics['durvieF'] # leaf lifespan from appearance in exponential phase
lifespan_early = sen_stics['ratiodurvieI'] * lifespan # leaf lifespan from appearance in juvenile phase

## 3. Set plant architectural parameters

Set topological, geometrical and developmental parameters, in a range corresponding a given species, found in literature.

In [ ]:
archi = dict(
    nb_phy=[8,20], # number of phytomers on the main stem
    nb_short_phy=4,
    
    # Stem
    height=3*max(height_canopy), # potential plant height
    stem_q=1.0, # parameter for ligule height geometric distribution along axis
    diam_base=2.5, # stem base diameter
    diam_top=1.5, # stem top diameter

    # Leaf area distribution along the stem  
    leaf_area=1.2*max(leaf_area_plant), # potential plant leaf area
    rmax=[0.55,0.8], # relative position of largest leaf on the stem
    skew=0.005, # skewness for leaf area distribution along axis

    # blade area
    wl=0.12, # leaf blade width-to-length ratio 
    klig=0.6, # parameter for leaf blade shape
    swmax=0.55, # relative position of maximal blade width
    f1=0.64, # parameter for leaf blade shape
    f2=0.92, # parameter for leaf blade shape

    # blade curvature
    insertion_angle=35, # leaf blade insertion angle
    scurv=0.7, #  relative position of inflexion point
    curvature=120, # leaf blade insertion-to-tip angle
    phyllotactic_angle=137.5, # phyllotactic angle
    phyllotactic_deviation=0, # half-deviation to phyllotactic angle

    # Development
    phyllochron=30, # phyllochron, i.e. stem element appearance rate
    plastochron=40, # plastochron, i.e. leaf blade appearance rate

    # Senescence 
    leaf_lifespan = [lifespan_early, lifespan], # leaf lifespan from appearance

    # Tillering
    nb_tillers=0, # number of tillers
    tiller_delay=1, # delay, as factor of phyllochron, between the appearance of a phytomer and the appearance of its tiller
    tiller_angle=30,
    reduction_factor=1, # reduction factor between tillers of consecutive order

    plant_orientation=0 #20
)

In [ ]:
# Function to process parameters and generate samples
# Generate parameter sets
param_sets = LHS_param_sampling(archi, daily_dynamics, n_samples=10, seed=18)
print(len(param_sets))

In [ ]:
# Select parameters sets for which the model fits the LAI and the height curves of the crop model, with a given error.

# Start the timer
start_time = t.time()

fitting_sim, non_fitting_sim = params_for_curve_fit(param_sets, curves=daily_dynamics, error_LA=0.05, error_height=0.05)

# End the timer
end_time = t.time()

# Calculate elapsed time
elapsed_time = (end_time - start_time)/60
print(f"Elapsed time: {elapsed_time:.4f} minutes")

In [ ]:
print(len(fitting_sim['mtg']))

In [ ]:
from matplotlib.lines import Line2D

# Create a figure with two subplots side by side
fig, axes = plt.subplots(2, 1, figsize=(12, 6), sharex=True)  # 1 row, 2 columns

# Plot on the first subplot
# for non_result in non_fitting_sim['LA']:
#     axes[0].plot(thermal_time, [r*sowing_density/10000 for r in result],  color="gray", alpha=0.1)
for result in fitting_sim['LA']:
    axes[0].plot(thermal_time, [r*sowing_density/10000 for r in result],  color="green", alpha=0.6)
    # print(result)
axes[0].plot(thermal_time, [(la-sen)*sowing_density/10000 for la, sen in zip(leaf_area_plant, sen_leaf_area_plant)], color="black", alpha=0.9)
# axes[0].set_xlabel("Thermal time (°C.day)")
axes[0].set_ylabel("LAI (m²/m²)", fontsize=16, fontname="Times New Roman")
# axes[0].set_title("Leaf Area: 3D canopy vs. STICS")
# axes[0].legend(loc=2)

legend_elements_lai = [
    Line2D([0], [0], color='black', alpha=0.9, lw=2, label='LAI STICS'),
    Line2D([0], [0], color='green', alpha=0.6, lw=2, label='LAI morphotypes')
]

axes[0].legend(handles=legend_elements_lai, loc=2, prop={'family': 'Times New Roman', 'size': 12})

# Plot on the second subplot
# for non_result in non_fitting_sim['height']:
#     axes[1].plot(thermal_time, [r*0.01 for r in non_result], color="gray", alpha=0.1)
for result in fitting_sim['height']:
    axes[1].plot(thermal_time, [r*0.01 for r in result], color="orange", alpha=0.6)
axes[1].plot(thermal_time, [h*0.01 for h in height_canopy], color="black", alpha=0.9)
axes[1].set_xlabel("Thermal time (°C.day)", fontsize=16, fontname="Times New Roman")
axes[1].set_ylabel("Crop height (m)", fontsize=16, fontname="Times New Roman")
# axes[1].set_title("Plant height: 3D canopy vs. STICS")

legend_elements_height = [
    Line2D([0], [0], color='black', alpha=0.9, lw=2, label='Height STICS'),
    Line2D([0], [0], color='orange', alpha=0.6, lw=2, label='Height morphotypes')
]

axes[1].legend(handles=legend_elements_height, loc=2, prop={'family': 'Times New Roman', 'size': 12})

plt.savefig('PMA_curves')

# Adjust layout
plt.tight_layout()

# Show the plot
plt.show()

In [ ]:
# fitting_sim['mtg'][-1][85].properties()

In [ ]:
scene, _ = build_scene(fitting_sim['mtg'][1][60])
PlantGL(scene)

In [ ]:
morpho = {
    i : [p['nb_phy'],p['rmax']]
    for i,p in enumerate(fitting_sim['params'])
}

# print(morpho)
# print([p['nb_phy'] for p in fitting_sim['params']])
# print([round(p['wl'],2) for p in fitting_sim['params']])

## Light interception

In [ ]:
import pandas as pd
from openalea.astk.sky_irradiance import sky_irradiance
from openalea.astk.sky_sources import sky_sources, caribu_light_sources
from openalea.archicrop.ltfs import illuminate, mean_leaf_irradiance
from alinea.caribu.data_samples import data_path

par_inc = [value["Incident PAR"] for value in daily_dynamics.values()]

fn = 'climaisj.meteo'
def meteo_day():
    names=['station', 'year', 'month', 'day', 'julian', 'min_temp', 'max_temp', 'rad', 'Penman PET', 'rainfall', 'wind', 'pressure', 'CO2']
    df = pd.read_csv(fn,  header=None, sep='\s+', names=names)
    df["daydate"] = pd.to_datetime(df[["year", "month", "day"]])
    return df

df = meteo_day()
location ={
'longitude': 3.87,
'latitude': 45,
'altitude': 56,
'timezone': 'Europe/Paris'}

# print([d for d in df.itertuples()])

# irr = sky_irradiance()
# sun, sky = sky_sources(sky_type='clear_sky', sky_irradiance=irr, scale='ppfd')
# lights = caribu_light_sources(sun, sky)

nice_green = Color3((50,100,0))

inter_plant = (100 / inter_row / sowing_density)*100 
domain = ((-0.5*inter_row, -0.5*inter_plant), (0.5*inter_row, 0.5*inter_plant))

par_caribu = []

scenes = []

start_time = t.time()

for mtgs in fitting_sim['mtg'][:1]:
    scenes_tmp = []
    aggs_tmp = []
    count = 0
    print(len(mtgs))
    # for mtg, par in zip(mtgs[-2:-1], par_inc[-2:-1]):
    for mtg,row in zip(mtgs,df.itertuples()):
        count += 1
        if count%5==0:
            irr = sky_irradiance(daydate=row.daydate, day_ghi=row.rad, **location)
            sun, sky = sky_sources(sky_type='clear_sky', sky_irradiance=irr, scale='global')
            lights = caribu_light_sources(sun, sky)
            # lights = [(par,(0,0,-1))]
            
            scene, labels = build_scene(mtg, (0,0,0), leaf_material=Material(nice_green), stem_material=Material(nice_green), senescence=False)
            cs, raw, agg = illuminate(scene, light=lights, labels=labels, domain=domain)
            aggs_tmp.append(agg)
            scenes_tmp.append(cs.plot(raw, display=False)[0])
            print((t.time() - start_time)/60)
    par_caribu.append(aggs_tmp)
    scenes.append(scenes_tmp)

end_time = t.time()

elapsed_time = (end_time - start_time)/60
print(f"Elapsed time: {elapsed_time:.4f} minutes for {len(fitting_sim['mtg'])} simulations")

In [ ]:
par_stics = [value["Absorbed PAR"] for value in daily_dynamics.values()]

nrj_per_leaf = []
irr_per_plant = []

for case in par_caribu:
    nrj_tmp = []
    irr_tmp = []
    for df in case:
        df_mod = mean_leaf_irradiance(df)
        nrj_tmp.append(df.loc[df['label'] == 'Leaf']['Energy'].values)
        irr_tmp.append(df_mod['Irradiance'].values[0])
    nrj_per_leaf.append(nrj_tmp)
    irr_per_plant.append(irr_tmp)

In [ ]:
nrj_per_plant = [[sum(growing_plant) for growing_plant in plant] for plant in nrj_per_leaf]
nrj_per_plant

In [ ]:
# Conversion
# irr_per_plant = [[i*0.0864/4.6  for i in irr] for irr in irr_per_plant]
irr_per_plant

In [ ]:
curves = nrj_per_plant
# curves_array = np.array(curves)

# # Calculate the envelope: min and max values for each time point
# min_values = curves_array.min(axis=0)
# max_values = curves_array.max(axis=0)

# Plotting the envelope along with individual curves for context
time_points = thermal_time
for curve in curves:
    # plt.plot(time_points, [i/par for i,par in zip(curve, par_inc)], alpha=0.5, linestyle='--')  # Plot each curve (optional for visualization)
    plt.plot([t for c,t in enumerate(time_points) if c%5==0 and c!=0], [i/par for i,par in zip(curve, [r for c,r in enumerate(par_inc) if c%5==0 and c!=0])], color='orange', label="ArchiCrop x Caribu")

# plt.fill_between(time_points, min_values, max_values, color="skyblue", alpha=0.4)
# plt.plot(time_points, min_values, color="blue", linestyle="--", label="Min 3D")
# plt.plot(time_points, max_values, color="red", linestyle="--", label="Max 3D")
plt.plot(time_points, par_stics, color="black", label="STICS")
# plt.scatter(time_points, LA_stics)

# Labels and legend
plt.xlabel("Thermal time")
plt.ylabel("Fraction of absorbed PAR")
plt.title("Absorbed PAR: 3D canopy vs. STICS")
plt.legend()
plt.show()

In [ ]:
# what about senescent parts ? they intercept light but do not absorb it

In [ ]:
PlantGL(scenes[0][-10])

In [ ]:
inter_row = 70
inter_plant = (100 / inter_row / sowing_density)*100 
print(inter_plant)
domain = ((-0.5*inter_row, -0.5*inter_plant), (0.5*inter_row, 0.5*inter_plant))
print(domain)

In [ ]:
sowing_density